---
title: "Diseño de bases de datos y recolección de información"
subtitle: "Seminario de Proyecto I (MCDI) — Unidad 4"
author: "Sergio Martín Nava Muñoz"
date: "`r format(Sys.Date(), '%d de %B de %Y')`"
format:
  revealjs:
    theme: [default]
    slide-number: true
    toc: true
    toc-depth: 2
    code-overflow: wrap
    incremental: true
    footer: "MCDI | Seminario de Proyecto I | Unidad 4"
execute:
  echo: false
  warning: false
  message: false
---


## Propósito de la sesión

- Entender los **principios de diseño de bases de datos (BD)** para proyectos de Ciencia de Datos.  
- Delinear un **plan de recolección de información** con criterios de calidad, ética y trazabilidad.  
- Preparar los insumos para la evidencia **4A. Creación de la base de datos** y la **presentación final** del protocolo.

::: callout-tip
**Productos esperados**  
1) Esquema lógico de BD (tablas/campos/llaves).  
2) Diccionario de datos.  
3) Plan de recolección y control de calidad.  
4) Muestra mínima de la BD con metadatos.
:::

## Ruta lógica del diseño de datos

1. **Problema / variables** → ¿Qué fenómeno quiero medir? (VD, VI, covariables)  
2. **Modelo conceptual** → Entidades, relaciones, cardinalidades.  
3. **Esquema lógico** → Tablas, claves primarias/foráneas, tipos de datos.  
4. **Normalización y performance** → 1FN–3FN vs. desnormalización táctica.  
5. **Gobernanza y metadatos** → diccionario, convenciones, control de versiones.  
6. **Recolección** → fuentes, instrumentos, muestreo, formato, ética.  
7. **Calidad** → validaciones, reglas de negocio, auditoría, reproducibilidad.

---

## De variables a entidades {.smaller}

- **Variable dependiente (VD)**: resultado a explicar/predcir.  
- **Variables independientes (VI)**: insumos/explicativas.  
- **Derivadas / indicadores**: transformaciones útiles para el análisis.

**Ejemplo** (fraude transaccional):  
- VD: `is_fraud` (0/1).  
- VI nucleares: `amount`, `channel`, `merchant_id`, `customer_id`, `timestamp`.  
- Derivadas: `hour`, `dayofweek`, `rolling_amount_1h`, `distance_home_merchant`.

::: columns
::: column
**Entidades**  
- `customers (customer_id, ...)`  
- `merchants (merchant_id, ...)`  
- `transactions (tx_id, customer_id, merchant_id, amount, ts, ...)`
:::
::: column
**Relaciones**  
- 1:N `customers` → `transactions`  
- 1:N `merchants` → `transactions`  
- Opcional: N:M con tablas puente si aplica
:::
:::

---

## Modelo conceptual (borrador ER)


```{mermaid}
erDiagram
  CUSTOMERS ||--o{ TRANSACTIONS : makes
  MERCHANTS ||--o{ TRANSACTIONS : receives
  CUSTOMERS {
    string customer_id PK
    string segment
    date   signup_date
  }
  MERCHANTS {
    string merchant_id PK
    string category
    string city
  }
  TRANSACTIONS {
    string tx_id PK
    string customer_id FK
    string merchant_id FK
    datetime ts
    numeric amount
    string channel
    bool   is_fraud
  }
```

> Diagrama de primer corte. Refínalo con tus variables reales.

---

## Esquema lógico y claves

- **Claves primarias (PK)**: identifican unívocamente filas.  
- **Claves foráneas (FK)**: preservan integridad referencial.  
- **Tipos de datos**: escoger por uso y performance (INTEGER, NUMERIC, TEXT, DATE/TIMESTAMP, BOOLEAN).  
- **Índices**: en columnas de `JOIN`, filtros frecuentes y fechas.

**Reglas rápidas**  
- PK **surrogada** (`serial`/`uuid`) cuando no existe una natural estable.  
- Evitar usar texto libre como FK.  
- Mantener **inmutables** las PK.

---

## Normalización práctica

- **1FN**: sin repetición de grupos; tipos atómicos.  
- **2FN**: no dependencias parciales en PK compuestas.  
- **3FN**: no dependencias transitivas.  

**Cuándo desnormalizar**  
- Lecturas analíticas muy frecuentes (p. ej., agregar columnas derivadas para *features*).  
- Tablas anchas para *feature store* (con versión y procedencia).

---

## Diccionario de datos (plantilla) {.smaller}

| Tabla | Variable | Tipo | Dominios/Reglas | Nulos | Descripción | Fuente | Transformación |
|---|---|---|---|---:|---|---|---|
| transactions | tx_id | TEXT | UUID v4 | 0 | Identificador de transacción | sistema | — |
| transactions | amount | NUMERIC(12,2) | ≥ 0 | 0 | Monto en MXN | POS | redondeo 2 dec |
| transactions | ts | TIMESTAMP | UTC | 0 | Fecha/hora de operación | POS | tz-normalizada |
| transactions | is_fraud | BOOLEAN | {0,1} | 0 | Etiqueta objetivo | analista | regla v1.2 |

> Mantén este archivo versionado (`data_dictionary.csv`) y sincronizado con el esquema.

---

## Recolección: fuentes e instrumentos

- **Fuentes primarias**: encuestas, sensores, logs, formularios.  
- **Fuentes secundarias**: bases gubernamentales, APIs, repos institucionales.  
- **Instrumentos**: formularios (con validación en captura), *ETL* programados, *web scraping* con cuidado ético-legal.

**Muestreo y cobertura**  
- Define unidad de análisis, marco muestral, tamaño y sesgos potenciales.  
- Estratificación cuando existan subpoblaciones clave.

---

## Calidad de datos (antes/durante/después)

**Prevención** (en captura)  
- Tipos y rangos en el formulario/DBMS.  
- Catálogos cerrados y validación cruzada.

**Detección y corrección** (ETL)  
- Reglas: unicidad de PK, dominios, consistencia temporal, *outliers* plausibles.  
- Bitácora de errores, *logs* de cambios, *unit tests* de datos.

**Trazabilidad**  
- Metadatos `source`, `ingestion_ts`, `version`, `checksum`.  
- Carpeta `raw/` (inmutable) → `staging/` → `curated/` (*medallion*).

---

## Ética, privacidad y datos sintéticos

- Minimiza la **recolección de datos sensibles** si no son imprescindibles.  
- Aplica anonimización/pseudonimización y controles de acceso.  
- Cuando compartir datos sea necesario, evalúa **datos sintéticos** para balancear **utilidad vs. riesgo de revelación** (por ejemplo, con técnicas basadas en árboles de decisión, *random forest* o redes neuronales).  
- Mide utilidad (p. ej., divergencias, solapamiento de ICs) y **riesgo de re-identificación** con escenarios de intrusor.

> Nota: en comparativas publicadas, los **árboles de decisión** suelen ser eficaces y eficientes para sintetizar datos con buen equilibrio entre utilidad y riesgo; las redes neuronales pueden reducir riesgo a costa de mayor tiempo de cómputo. Sustenta con métricas y reporte metodológico.

---

## Arquitectura mínima de trabajo (proyecto)

```
/data
  /raw           # fuentes originales solo lectura
  /staging       # limpieza/estandarización
  /curated       # listo para análisis/modelado
  data_dictionary.csv
/db
  schema.sql     # DDL y constraints
/notebooks
  01_exploracion.ipynb
  02_validaciones.ipynb
/src
  etl_ingesta.py
  reglas_calidad.yml
README.md
```

---

## Mini-guía para DBMS

- **SQLite**: prototipado local, *lightweight*, perfecto para empezar.  
- **PostgreSQL**: robusto, extensiones GIS/JSONB, ideal producción.  
- **DuckDB**: analítica en archivos Parquet/CSV, rápido en local.  
- **NoSQL** (cuando aplica): eventos/clave-valor, documentos o grafos.

> Elige según concurrencia, volumen, tipo de consultas y facilidad de despliegue.

---

## Checklist de entrega (Evidencia 4A)

- [ ] Archivo de base (`.db`, `.parquet`, `.csv` o similar) y **tamaño**.  
- [ ] DBMS utilizado (SQLite/PostgreSQL/DuckDB/otro).  
- [ ] **Variables** incluidas y número de observaciones.  
- [ ] **Diccionario** actualizado.  
- [ ] **Video** (≤ 3 min) mostrando archivo, peso, DBMS y contenido.  
- [ ] PDF con **enlace al video** y captura de pantalla.

---

## Actividad guiada (15–20 min)

1) Dibuja el **ER** de tus datos (3–5 entidades).  
2) Define **PK/FK** y tipa 10 variables clave.  
3) Escribe 6 reglas de **calidad** (3 prevención, 3 detección).  
4) Redacta **plan de recolección** (fuentes, instrumento, muestreo, ética).

> Entregable: subir a la carpeta del curso tu ER (imagen), `schema.sql` y `data_dictionary.csv`.

---

## Apéndice — Esqueleto `schema.sql`


```{sql}
-- Tabla principal de transacciones
CREATE TABLE transactions (
  tx_id TEXT PRIMARY KEY,
  customer_id TEXT NOT NULL,
  merchant_id TEXT NOT NULL,
  ts TIMESTAMP NOT NULL,
  amount NUMERIC(12,2) CHECK (amount >= 0),
  channel TEXT CHECK (channel IN ('pos','web','app','ivr')),
  is_fraud BOOLEAN NOT NULL DEFAULT 0,
  -- metadatos
  source TEXT,
  ingestion_ts TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE INDEX idx_transactions_ts ON transactions(ts);
CREATE INDEX idx_transactions_customer ON transactions(customer_id);

-- Catálogos
CREATE TABLE customers (
  customer_id TEXT PRIMARY KEY,
  segment TEXT,
  signup_date DATE
);

CREATE TABLE merchants (
  merchant_id TEXT PRIMARY KEY,
  category TEXT,
  city TEXT
);

ALTER TABLE transactions
  ADD FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
  ADD FOREIGN KEY (merchant_id) REFERENCES merchants(merchant_id);
```


---

## Referencias mínimas

- Dandekar, A., Zen, R. A. M., & Bressan, S. (2018). *A Comparative Study of Synthetic Dataset Generation Techniques*. DEXA 2018, 387–395.  
- Ascolano Ruiz, F., Cazorla Quevedo, M. A., Alfonso, M. I., Colomina Pardo, O., & Lozano Ortega, M. A. (2003). *Inteligencia Artificial: modelos, técnicas y áreas de aplicación*. Paraninfo.  
- Baesens, B. (2014). *Analytics in a Big Data World*. Wiley.  
- Lateh, M. A., et al. (2017). Handling a small dataset problem in prediction model by employ artificial data generation approach. *J. Phys.: Conf. Ser.*, 892(1), 012016.

---

## Cierre

- La calidad del **diseño de datos** determina la calidad del **análisis**.  
- Documenta y automatiza: **diccionario + DDL + ETL + pruebas**.  
- Evalúa la opción de **datos sintéticos** si debes compartir o abrir datos.  
- Asegura **ética, privacidad y reproducibilidad** desde el día 1.